# Segment Anything Model Finetuning for Remote Sensing Applications


In this project you will be asked to segment sidewalks from satellite imagery. The importance of remote sensing and its associated job market is growing rapidly. Remote sensing powers, weather prediction, agriculture, urban planning, defense and many other applications. 

Lately, we have observed the advent of very large foundational models in NLP applications - sometimes these models result in  hundreds of billions of parameters - for example the [GPT-3](https://arxiv.org/abs/2005.14165) model that has 175 billion parameters. Can we do the same for computer vision applications ?  An example of a positive answer to this problem is the Segmenting Anything Model (SAM).  

## Background

![](images/sam.png)

Read and experiment with [the SAM implementation](https://segment-anything.com/) and [the SAM paper](https://arxiv.org/abs/2304.02643).


## Milestone 1: Get to know Geographical Information Systems (GIS) and set up your development environment

See [this tutorial](https://docs.qgis.org/3.34/en/docs/gentle_gis_introduction/index.html) that is best to be followed in a desktop installation of [QGIS](https://qgis.org/en/site/). 

Use the [example docker container](https://github.com/pantelis/artificial-intelligence) if you have access to a local NVIDIA GPU. Note that you need to do the project in Pytorch and you need to edit the `devcontainer.json` file to point to the right container for PyTorch.   

In addition to python scripts you will also need python notebooks to experiment and visualize the SAM model outputs as well as geospatial data. The suggested container ships with [Jupyter](https://jupyter.org/) and if you use [Google Colab](https://colab.research.google.com/) you are obviously using notebooks.  

ScreenShots

[](./images/Container.png)

[](./images/qgis.png)

## Milestone 2: Replicate SAM implementation for satellite imagery

The SAM implementation for remote sensing applications is based on [this package](https://samgeo.gishub.org/). Go over the following video and replicate the workflow in your environment for the imagery used by the author.    

https://www.youtube.com/watch?v=YHA_-QMB8_U

The notebook based environment makes it very easy to experiment with the SAM model and the workflow. 

This [desktop QGIS plugin](https://github.com/BjornNyberg/Geometric-Attributes-Toolbox/wiki/User-Guide#segment-anything-model) is also available to help you experiment with the SAM model.